# Differentiable Boolean Satisfiability

This notebook demonstrates a differentiable implementation of the [Boolean Satisfiability Problem](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem) (SAT), encoded in [Conjunctive Normal Form](https://en.wikipedia.org/wiki/Conjunctive_normal_form) (CNF).

## Key Concept

The SAT problem asks: given a boolean formula, is there an assignment of true/false values to variables that makes the formula evaluate to true? Traditionally, this is a discrete optimization problem, but by using continuous relaxations, we can apply gradient-based optimization.

**Encoding Convention:**
- $True$ is encoded as floating point $1.0$  
- $False$ is encoded as floating point $0.0$

This allows us to use standard neural network optimization techniques to find satisfying assignments!

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

## Setup

Let's import the necessary libraries and verify our TensorFlow setup:

In [2]:
tf.executing_eagerly()

True

In [ ]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Differentiable Logic Gates

### Differentiable AND Gate

Using floating point multiplication as an approximation for the $AND$ gate. This works perfectly for our boolean encoding:

$$0.0 \times 1.0 = 0.0 \quad (\text{False AND True} = \text{False})$$
$$1.0 \times 1.0 = 1.0 \quad (\text{True AND True} = \text{True})$$

**Key insight:** Multiplication naturally implements AND logic when inputs are constrained to $\{0,1\}$, and the gradients provide useful optimization signals even for intermediate values.

In [4]:
# https://github.com/tensorflow/tensorflow/issues/29781#issuecomment-542496509
def compute_hessian(f, *x):
    grad_grads = []
    with tf.GradientTape(persistent=True) as hess_tape:
        with tf.GradientTape() as grad_tape:
            y = f(*x)
        grad = grad_tape.gradient(y, x)
        for g in grad:
            grad_grads += list(hess_tape.gradient(g, x))
    hessian = tf.stack(grad_grads)
    return hessian

### Differentiable NOT Gate

Since valid boolean values are constrained between $0.0$ and $1.0$, we can implement negation by subtracting from $1.0$:

$$1.0 - 0.0 = 1.0 \quad (\text{NOT False} = \text{True})$$
$$1.0 - 1.0 = 0.0 \quad (\text{NOT True} = \text{False})$$

**Note:** The gradient is always $-1$, which provides a clear optimization signal.

In [ ]:
@tf.function
def gate_and(x, y):
    return x * y


x = tf.Variable([1, 0, 1, 0], dtype=tf.float32)
y = tf.Variable([1, 1, 0, 0], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    z = gate_and(x, y)

print(z)
print(tape.gradient(z, x))
print(tape.gradient(z, y))
print(compute_hessian(gate_and, x, y))

tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
tf.Tensor([1. 1. 0. 0.], shape=(4,), dtype=float32)
tf.Tensor([1. 0. 1. 0.], shape=(4,), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]], shape=(4, 4), dtype=float32)


### Differentiable OR Gate

We compose the OR gate using [De Morgan's Law](https://en.wikipedia.org/wiki/De_Morgan's_laws):

$$ X \lor Y = \overline{\overline{X} \land \overline{Y}} $$

This is elegant because it reuses our existing AND and NOT implementations, ensuring consistency across all logic operations.

In [ ]:
@tf.function
def gate_not(x):
    return 1 - x


x = tf.Variable([1, 0, 1, 0], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    z = gate_not(x)

print(z)
print(tape.gradient(z, x))

tf.Tensor([0. 1. 0. 1.], shape=(4,), dtype=float32)
tf.Tensor([-1. -1. -1. -1.], shape=(4,), dtype=float32)


### Differentiable XOR Gate

We implement XOR from its definition as "exclusive or":

$$ X \oplus Y = (\overline{X} \land Y) \lor (X \land \overline{Y}) $$

XOR is true when exactly one input is true. This gate will be crucial for our SAT solver implementation, as it helps us determine when variables match their expected polarities in clauses.

In [ ]:
@tf.function
def gate_or(x, y):
    not_x = gate_not(x)
    not_y = gate_not(y)

    x_not_y_not = gate_and(not_x, not_y)
    x_or_y = gate_not(x_not_y_not)

    return x_or_y


x = tf.Variable([1, 0, 1, 0], dtype=tf.float32)
y = tf.Variable([1, 1, 0, 0], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    z = gate_or(x, y)

print(z)
print(tape.gradient(z, x))
print(tape.gradient(z, y))

tf.Tensor([1. 1. 1. 0.], shape=(4,), dtype=float32)
tf.Tensor([-0. -0.  1.  1.], shape=(4,), dtype=float32)
tf.Tensor([-0.  1. -0.  1.], shape=(4,), dtype=float32)


## SAT Problem Encoding

### Candidate Encoding

A **candidate** represents a potential solution - an assignment of truth values to variables. For example, if we have variables $X_1, X_2, X_3$, then the assignment $X_1 = False$, $X_2 = True$, $X_3 = False$ is encoded as:

\begin{equation*}
\text{Candidate} = 
\begin{bmatrix}
0.0 & 1.0 & 0.0 \\
\end{bmatrix}\end{equation*}

### Circuit Encoding

The boolean formula in CNF is encoded as a 2D tensor where each row represents a clause (disjunction) and each column represents a variable. The value indicates whether the variable appears negated in that clause.

For example, the formula:
$$
(X_1 \lor X_2 \lor \overline{X_3}) \land
(X_1 \lor \overline{X_2} \lor X_3) \land 
(\overline{X_1} \lor X_2 \lor X_3)
$$

is encoded as:

\begin{equation*}
\text{Circuit} = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 \\  \text{  ← } (X_1 \lor X_2 \lor \overline{X_3}) \\
0.0 & 1.0 & 0.0 \\  \text{  ← } (X_1 \lor \overline{X_2} \lor X_3) \\
1.0 & 0.0 & 0.0     \text{  ← } (\overline{X_1} \lor X_2 \lor X_3)
\end{bmatrix}\end{equation*}

where $1.0$ indicates the variable appears negated in that clause, $0.0$ indicates it appears positive.

### Efficient GPU Algorithm

To maximize GPU utilization, we use a parallel evaluation strategy:

**Step 1: Broadcasting**  
First, we broadcast/tile the candidate to match the circuit dimensions:

\begin{equation*}
\text{Candidate} = 
\begin{bmatrix}
0.0 & 1.0 & 0.0 \\
\end{bmatrix}\end{equation*}

becomes (repeated for each clause):

\begin{equation*}
\text{Candidate}_{broadcasted} = 
\begin{bmatrix}
0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 \\
\end{bmatrix}\end{equation*}

**Step 2: Flattening**  
We flatten the circuit encoding:

\begin{equation*}
\text{Circuit}_{flat} = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 &
0.0 & 1.0 & 0.0 &
1.0 & 0.0 & 0.0
\end{bmatrix}\end{equation*}

**Step 3: XOR Operation**  
The XOR between candidate and circuit determines variable polarities. This efficiently computes whether each variable-clause pair contributes positively to satisfaction:

- XOR$(0,0) = 0$: Variable is False and appears positive → contributes False  
- XOR$(0,1) = 1$: Variable is False and appears negated → contributes True
- XOR$(1,0) = 1$: Variable is True and appears positive → contributes True  
- XOR$(1,1) = 0$: Variable is True and appears negated → contributes False

**Step 4: Parallel Reduction**  
Finally, we perform parallel reductions:
1. OR within each clause (any True literal satisfies the clause)
2. AND across all clauses (all clauses must be satisfied)

This approach leverages GPU parallelism for efficient SAT evaluation!

## Circuit Graph

### Candidate Encoding

Candidate is the input to the circuit. The satisfiability problem is considered solved when we find a candidate that makes the circuit output $True$.

Candidate $X_1 = 0$, $X_2 = 1$ and $X_3 = 0$ is encoded as tensor.

\begin{equation*}
Candidate = 
\begin{bmatrix}
0.0 & 1.0 & 0.0 \\
\end{bmatrix}\end{equation*}

### Circuit Encoding

Circuit is encoded as a 2D tensor. e.g.

$$
(X_1 \lor X_2 \lor \overline{X_3}) \land
(X_1 \lor \overline{X_2} \lor X_3) \land 
(\overline{X_1} \lor X_2 \lor X_3)
$$

is encoded as

\begin{equation*}
Circuit = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 \\
0.0 & 1.0 & 0.0 \\
1.0 & 0.0 & 0.0
\end{bmatrix}\end{equation*}


## Constraint/Boundary Loss Function

Since we're working with continuous values but need binary solutions, we need a constraint loss that pushes values toward $0.0$ or $1.0$. 

This loss function has the following properties:
- **Minimum at 0 and 1**: $f(0) = f(1) = 0$ 
- **Maximum at 0.5**: $f(0.5) = 0.0625$
- **Steep penalties**: Values far from $\{0,1\}$ incur large penalties
- **Smooth gradients**: Provides optimization signals throughout $[0,1]$

### Algorithm

To make the best use of GPU we follow this method.

First we broadcast/tile the candidate to match the dimensions of the problem.

\begin{equation*}
Candidate = 
\begin{bmatrix}
0.0 & 1.0 & 0.0 \\
\end{bmatrix}\end{equation*}

becomes

\begin{equation*}
Candidate_{broadcasted} = 
\begin{bmatrix}
0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 \\
\end{bmatrix}\end{equation*}

Next we flatten the circuit

\begin{equation*}
Circuit = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 &
0.0 & 1.0 & 0.0 &
1.0 & 0.0 & 0.0
\end{bmatrix}\end{equation*}

The XOR of the broadcasted candidate and the flat circuit gives us the activations at each element. e.g.

$$
(X_1 \lor X_2 \lor \overline{X_3}) \land
(X_1 \lor \overline{X_2} \lor X_3) \land 
(\overline{X_1} \lor X_2 \lor X_3)
$$
$$=$$
$$
(0.0 \lor 1.0 \lor 1.0) \land
(0.0 \lor 0.0 \lor 0.0) \land 
(1.0 \lor 1.0 \lor 0.0)
$$

We shall now reshape the activations into 2D for the sake of clarity

\begin{equation*}
Activations = 
\begin{bmatrix}
0.0 & 1.0 & 1.0 \\
0.0 & 0.0 & 0.0 \\
1.0 & 1.0 & 0.0
\end{bmatrix}\end{equation*}

Now we parallel reduce the $\lor$ (OR) operations. We get

\begin{equation*}
Activations = 
\begin{bmatrix}
1.0 \\
0.0 \\
1.0
\end{bmatrix}\end{equation*}

On reducing the $\land$ (AND) operations, we get the output of the circuit

\begin{equation*}
Activations = 
\begin{bmatrix}
0.0
\end{bmatrix}\end{equation*}

Since, the whole thing is differentiable end to end. We can optimize using SGD.


In [ ]:
@tf.function
def circuit_forward_pass(candidate, problem_encoding):
    disjunction_count = tf.shape(problem_encoding)[0]

    broadcasted_candidate = tf.tile(candidate, [disjunction_count])
    reshaped_problem = tf.reshape(problem_encoding, [-1])

    resolved_values = gate_xor(broadcasted_candidate, reshaped_problem)
    reshaped_values = tf.reshape(resolved_values, [-1, disjunction_count])
    reshaped_values = tf.transpose(reshaped_values, [1, 0])

    conjunctions = tf.scan(gate_or, reshaped_values)[-1]
    conjunctions = tf.reshape(conjunctions, [-1, 1])
    disjunctions = tf.scan(gate_and, conjunctions)[-1][0]

    return disjunctions


candidate = tf.Variable([0, 1, 0], dtype=tf.float32)
problem_encoding = tf.Variable(
    [
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ],
    dtype=tf.float32,
)

with tf.GradientTape(persistent=True) as tape:
    z = circuit_forward_pass(candidate, problem_encoding)

print(z)
print(tape.gradient(z, candidate))
print(tape.gradient(z, problem_encoding))

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor([ 1. -1.  1.], shape=(3,), dtype=float32)
tf.Tensor(
[[ 0.  0.  0.]
 [ 1. -1.  1.]
 [ 0.  0.  0.]], shape=(3, 3), dtype=float32)


## Combined Optimization Loss

Our total loss function combines two objectives:

1. **Satisfiability Loss**: Drives the circuit output toward $1.0$ (satisfied)
2. **Constraint Loss**: Pushes variable assignments toward binary values $\{0,1\}$

This multi-objective approach finds solutions that both satisfy the boolean formula and have crisp binary assignments.

In [ ]:
@tf.function
def constraint_loss_fn(candidate):
    x = candidate
    a = x**2
    b = (x - 1) ** 2

    return a * b


candidate = tf.Variable([-0.5, 0, 0.25, 0.5, 0.75, 1, 1.5], dtype=tf.float32)
with tf.GradientTape() as tape:
    z = constraint_loss_fn(candidate)
    print(list(z.numpy()))
    print(list(tape.gradient(z, candidate).numpy()))

[0.5625, 0.0, 0.03515625, 0.0625, 0.03515625, 0.0, 0.5625]
[-3.0, 0.0, 0.1875, 0.0, -0.1875, 0.0, 3.0]


## Training/Optimization

Now we can train our differentiable SAT solver! We start with an initial guess for the variable assignments and use Adam optimizer to find a satisfying assignment.

The training process shows:
- **Loss decreasing**: The solver is finding better assignments
- **Variables converging**: Values approach binary $\{0,1\}$ solutions  
- **Circuit satisfaction**: Output approaches $1.0$ (fully satisfied)

## Optimization loss
Loss to make sure that the output is equal to 1.0

## Verification

Let's verify our solution! The optimizer found $X_1 = 1.0$, $X_2 = 1.0$, $X_3 = 1.0$ as a satisfying assignment.

**Manual verification:**
- Clause 1: $(1 \lor 1 \lor \overline{1}) = (1 \lor 1 \lor 0) = 1$ ✓
- Clause 2: $(1 \lor \overline{1} \lor 1) = (1 \lor 0 \lor 1) = 1$ ✓  
- Clause 3: $(\overline{1} \lor 1 \lor 1) = (0 \lor 1 \lor 1) = 1$ ✓

All clauses are satisfied, so our differentiable SAT solver successfully found a valid solution!

## Training/Optimization

In [ ]:
candidate = tf.Variable([0, 1, 0], dtype=tf.float32)
problem_encoding = tf.Variable(
    [
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ],
    dtype=tf.float32,
)

opt = tf.keras.optimizers.Adam(learning_rate=3e-4)


@tf.function
def train_step():
    with tf.GradientTape() as tape:
        z = circuit_forward_pass(candidate, problem_encoding)
        loss = loss_fn(candidate, z)
    grads = tape.gradient(loss, candidate)
    opt.apply_gradients(zip([grads], [candidate]))
    return loss


for i in range(10000):
    loss = train_step()
    if i % 1000 == 0:
        tf.print(i, loss, candidate)

0 1 [0.000300008513 0.9997 0.000300008513]
1000 0.372614384 [0.202071369 0.809448719 0.202071369]
2000 0.302438021 [0.317299336 0.773605466 0.317299306]
3000 0.217134744 [0.467763871 0.900396705 0.467763722]
4000 0.101519562 [0.686918437 1.02080131 0.686917782]
5000 0.012053174 [0.915573061 1.00370216 0.915572584]
6000 2.50427984e-05 [0.996464252 1.00000632 0.996464193]
7000 2.62632721e-10 [0.999989033 1.00000441 0.999989033]
8000 2.00159576e-11 [0.999997437 1.00000262 0.999997437]
9000 7.20489597e-12 [0.99999845 1.00000155 0.99999845]


## Result
$X_1 = 1.0$, $X_2 = 1.0$ and $X_3 = 1.0$ is a valid candidate which we can verify manually.

In [14]:
print(tf.round(candidate))
z = circuit_forward_pass(tf.round(candidate), problem_encoding)
print(z)

tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
